In [1]:
import sys
!{sys.executable} -m pip install pulp
!{sys.executable} -m pip install html5lib
import unittest
import pulp
import pandas as pd
import numpy as np
from pulp import LpVariable, LpProblem, LpMaximize, value
from typing import Tuple, Dict
from concurrent.futures import ThreadPoolExecutor


**Aggregating LMP DAM Prices for 2022**


In [2]:

# List of file paths or dataset names (replace these with your actual file paths)
file_names = [
    "Data/2022/Jan2022LMPDAM.csv",
    "Data/2022/Feb2022LMPDAM.csv",
    "Data/2022/Mar2022LMPDAM.csv",
    "Data/2022/Apr2022LMPDAM.csv",
    "Data/2022/May2022LMPDAM.csv",
    "Data/2022/Jun2022LMPDAM.csv",
    "Data/2022/Jul2022LMPDAM.csv",
    "Data/2022/Aug2022LMPDAM.csv",
    "Data/2022/Sep2022LMPDAM.csv",
    "Data/2022/Oct2022LMPDAM.csv",
    "Data/2022/Nov2022LMPDAM.csv",
    "Data/2022/Dec2022LMPDAM.csv"
]

# Initialize an empty list to store individual DataFrames
dataframes = []

# Loop through the file names, read each dataset, and append it to the list
for file in file_names:
    df = pd.read_csv(file)
    dataframes.append(df)

# Concatenate all DataFrames into a single DataFrame
LMPDAM_prices = pd.concat(dataframes, ignore_index=True)
LMPDAM_prices['datetime'] = pd.to_datetime(LMPDAM_prices['INTERVALSTARTTIME_GMT']).dt.tz_localize(None)
LMPDAM_prices = LMPDAM_prices.sort_values(by='datetime', ascending=True)
LMPDAM_prices = LMPDAM_prices.drop(columns=[
    'INTERVALSTARTTIME_GMT', 'INTERVALENDTIME_GMT', 'OPR_DT', 'OPR_HR', 'OPR_INTERVAL', 
    'NODE_ID_XML', 'NODE_ID', 'MARKET_RUN_ID', 'XML_DATA_ITEM', 'PNODE_RESMRID', 
    'GRP_TYPE', 'POS','GROUP','NODE'
])
LMPDAM_prices = LMPDAM_prices[LMPDAM_prices['LMP_TYPE'] == "LMP"]

LMPDAM_prices.head()



,LMP_TYPE,MW,datetime
0,LMP,66.64796,2022-01-01 07:00:00
26,LMP,62.65649,2022-01-01 08:00:00
11,LMP,64.72469,2022-01-01 09:00:00
9,LMP,60.83403,2022-01-01 10:00:00
5,LMP,61.59236,2022-01-01 11:00:00


**Wind Data and Feature Engineering**

In [3]:
#Load Weather data: https://data.nrel.gov/submissions/54

weather_df=pd.read_csv("Data/2022WindData.csv", skiprows=1) #For Longitude -123.99065 Latitude 38.350143
weather_df['datetime'] = pd.to_datetime(weather_df[['Year', 'Month', 'Day', 'Hour']])
weather_df = weather_df[weather_df['datetime'] >= '2022-01-01 07:00:00']

#Gas constant for dry air (in J/(kg.K))
R = 287.058

#Convert temperature from celsius to Kelvin
weather_df['air temperature at 100m (K)'] = weather_df['air temperature at 100m (C)'] + 273.15
weather_df['air temperature at 200m (K)'] = weather_df['air temperature at 200m (C)'] + 273.15
weather_df['air temperature at 300m (K)'] = weather_df['air temperature at 300m (C)'] + 273.15

# Calculate air density (ρ = p / (R * T))
weather_df['air density at 100m (kg/m³)'] = weather_df['air pressure at 100m (Pa)'] / (R * weather_df['air temperature at 100m (K)'])
weather_df['air density at 200m (kg/m³)'] = weather_df['air pressure at 200m (Pa)'] / (R * weather_df['air temperature at 200m (K)'])
weather_df['air density at 300m (kg/m³)'] = weather_df['air pressure at 300m (Pa)'] / (R * weather_df['air temperature at 300m (K)'])


**Wind Turbine Specifications and Power Output**

The output of a wind turbine depends on wind velocity, but power is not directly proportional to it. Each turbine has a unique power curve, which is used to determine its output at specific wind speeds. The power curve for the Northwind 100C will be used as a placeholder.

In [4]:
# Constants
Cp = 0.4  # Example coefficient of performance (adjust based on your turbine specifications)
R = 50  # Example blade length in meters (adjust based on your turbine specifications)

# Calculate the swept area of the wind turbine
A = np.pi * R**2

# Calculate power at each height using the formula P = 0.5 * Cp * ρ * A * V^3
weather_df['power at 100m (MW)'] = (
    0.5 * Cp * weather_df['air density at 100m (kg/m³)'] * A * (weather_df['wind speed at 100m (m/s)']**3) / 1e6
)
weather_df['power at 200m (MW)'] = (
0.5 * Cp * weather_df['air density at 200m (kg/m³)'] * A * (weather_df['wind speed at 200m (m/s)']**3) / 1e6
)
weather_df['power at 300m (MW)'] = (
    0.5 * Cp * weather_df['air density at 300m (kg/m³)'] * A * (weather_df['wind speed at 300m (m/s)']**3) / 1e6
)


weather_df.head()

,Year,Month,Day,Hour,Minute,air pressure at 100m (Pa),air pressure at 200m (Pa),air pressure at 300m (Pa),air temperature at 100m (C),air temperature at 120m (C),...,datetime,air temperature at 100m (K),air temperature at 200m (K),air temperature at 300m (K),air density at 100m (kg/m³),air density at 200m (kg/m³),air density at 300m (kg/m³),power at 100m (MW),power at 200m (MW),power at 300m (MW)
7,2022,1,1,7,30,101140,99920,98720,8.45,8.25,...,2022-01-01 07:00:00,281.60,280.61,279.64,1.251182,1.240451,1.229805,2.380741,2.468205,2.453813
8,2022,1,1,8,30,101170,99940,98740,8.48,8.28,...,2022-01-01 08:00:00,281.63,280.64,279.66,1.251420,1.240567,1.229966,1.794065,1.856630,1.846383
9,2022,1,1,9,30,101190,99980,98770,8.53,8.34,...,2022-01-01 09:00:00,281.68,280.70,279.73,1.251445,1.240798,1.230032,1.428272,1.463904,1.441644
10,2022,1,1,10,30,101280,100060,98850,8.58,8.38,...,2022-01-01 10:00:00,281.73,280.75,279.78,1.252336,1.241570,1.230808,1.003416,1.013582,0.986167
11,2022,1,1,11,30,101310,100090,98870,8.71,8.51,...,2022-01-01 11:00:00,281.86,280.88,279.91,1.252129,1.241367,1.230485,0.737182,0.743078,0.724440


In [5]:
merged_df = pd.merge(weather_df, LMPDAM_prices, on='datetime', how='inner')
merged_df.head()



,Year,Month,Day,Hour,Minute,air pressure at 100m (Pa),air pressure at 200m (Pa),air pressure at 300m (Pa),air temperature at 100m (C),air temperature at 120m (C),...,air temperature at 200m (K),air temperature at 300m (K),air density at 100m (kg/m³),air density at 200m (kg/m³),air density at 300m (kg/m³),power at 100m (MW),power at 200m (MW),power at 300m (MW),LMP_TYPE,MW
0,2022,1,1,7,30,101140,99920,98720,8.45,8.25,...,280.61,279.64,1.251182,1.240451,1.229805,2.380741,2.468205,2.453813,LMP,66.64796
1,2022,1,1,8,30,101170,99940,98740,8.48,8.28,...,280.64,279.66,1.251420,1.240567,1.229966,1.794065,1.856630,1.846383,LMP,62.65649
2,2022,1,1,9,30,101190,99980,98770,8.53,8.34,...,280.70,279.73,1.251445,1.240798,1.230032,1.428272,1.463904,1.441644,LMP,64.72469
3,2022,1,1,10,30,101280,100060,98850,8.58,8.38,...,280.75,279.78,1.252336,1.241570,1.230808,1.003416,1.013582,0.986167,LMP,60.83403
4,2022,1,1,11,30,101310,100090,98870,8.71,8.51,...,280.88,279.91,1.252129,1.241367,1.230485,0.737182,0.743078,0.724440,LMP,61.59236


In [6]:
def optimize_energy(merged_df: pd.DataFrame, params: Dict, storage_capacity: float) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Optimize energy allocation for wind power based on DAM prices, storage capacity, private market demand, etc.

    Args:
        merged_df (pd.DataFrame): DataFrame with wind power, DAM prices, and other relevant data.
        params (dict): Parameters for the optimization model.
        storage_capacity (float): Maximum storage capacity of the battery (MWh).

    Returns:
        Tuple[pd.DataFrame, pd.DataFrame]: A tuple containing the results DataFrame and a summary DataFrame.
    """
    writeMPS=False
    
    # Extract parameters
    power_at_height = params.get('power_at_height', 'power at 100m (MW)')
    wind_power = merged_df[power_at_height].values * params['n']
    dam_prices = merged_df['MW'].values
    private_market_demand = params['private_market_demand']
    private_price = params['private_price']
    storage_efficiency = params['storage_efficiency']
    charge_C_rate = params['charge_C_rate']
    discharge_C_rate = params['discharge_C_rate']
    hours = range(len(wind_power))

    # Define decision variables
    sell_private = pulp.LpVariable.dicts("SellPrivate", hours, lowBound=0, cat='Continuous')
    sell_grid = pulp.LpVariable.dicts("SellGrid", hours, lowBound=0, cat='Continuous')
    charge = pulp.LpVariable.dicts("Charge", hours, lowBound=0, upBound=charge_C_rate*storage_capacity, cat='Continuous')
    discharge_grid = pulp.LpVariable.dicts("DischargeGrid", hours, lowBound=0, upBound=discharge_C_rate*storage_capacity, cat='Continuous')
    stored_energy = pulp.LpVariable.dicts("StoredEnergy", hours, lowBound=0, upBound=storage_capacity, cat='Continuous')
    discharge_private = pulp.LpVariable.dicts("DischargePrivate", hours, lowBound=0, upBound=discharge_C_rate*storage_capacity, cat='Continuous')

    # Define the optimization problem
    prob = LpProblem("Energy_Optimization", LpMaximize)

    # Objective function: Maximize profit
    prob += pulp.lpSum(
            dam_prices[h] * sell_grid[h] +
            private_price * sell_private[h] +
            dam_prices[h] * discharge_grid[h] +
            private_price * discharge_private[h] -
            charge[h] * storage_efficiency  # Penalize for inefficiency
            for h in hours
            )
    
    # Constraints
    for h in hours:
        
        prob += charge[h] >= 0 
        prob += sell_grid[h] >= 0
        prob += discharge_grid[h] >= 0 
        prob += discharge_private[h] >= 0
        prob += charge[h] <= storage_capacity     
        prob += charge[h] <= storage_capacity * charge_C_rate
        prob += charge[h] <= wind_power[h] - sell_private[h] - sell_grid[h] 
        prob += discharge_grid[h] + discharge_private[h] <= stored_energy[h] 
        prob += sell_private[h] +discharge_private[h]== private_market_demand 
        prob += sell_grid[h] <= wind_power[h] - sell_private[h] + discharge_grid[h] 
        prob += discharge_grid[h] + discharge_private[h] <= storage_capacity * discharge_C_rate
        prob += sell_private[h] <= wind_power[h]-sell_grid[h] + (stored_energy[h]-discharge_grid[h])  

    # Storage dynamics
    for h in hours: 
        if h == 0:
            prob += stored_energy[h] == charge[h]*storage_efficiency  - discharge_grid[h] - discharge_private[h]
        else:
            prob += stored_energy[h] == stored_energy[h-1] + charge[h]*storage_efficiency  - discharge_grid[h] - discharge_private[h]


        

    # Solve the problem
    prob.solve()

    #Check if solution is optimal
    if pulp.LpStatus[prob.status] != "Optimal":
        raise ValueError(f"Solver status is {pulp.LpStatus[prob.status]}")

    # Extract Decision Results   
    results =  pd.DataFrame({
        "Hour": hours,
        "Energy Sold to Private Market": [sell_private[h].varValue for h in hours],
        "Energy Discharged to Private Market": [discharge_private[h].varValue for h in hours],
        "Energy Sold to Grid": [sell_grid[h].varValue for h in hours],
        "Energy Charged": [charge[h].varValue for h in hours],
        "Energy Discharged to Grid": [discharge_grid[h].varValue for h in hours],
        "Energy Stored": [stored_energy[h].varValue for h in hours],
        "Wind Power Generated": wind_power,
        "LMP DAM Price": dam_prices})
    
    results_df = pd.DataFrame(results)
    
    # Financial summary
    total_revenue = (
        (results_df['Energy Sold to Private Market'] * private_price).sum() +
        (results_df['Energy Discharged to Private Market'] * private_price).sum() +
        (results_df['Energy Sold to Grid'] * results_df['LMP DAM Price']).sum() +
        (results_df['Energy Discharged to Grid'] * results_df['LMP DAM Price']).sum()
        )   
    
    unoptimized_revenue =sum(
        (params['private_market_demand'] * params['private_price']) + 
        max(0, (results_df["Wind Power Generated"][h] - private_market_demand)) * dam_prices[h] 
        for h in range(len(merged_df))
        )
    
    total_production = results_df['Energy Sold to Private Market'].sum() + results_df['Energy Sold to Grid'].sum()
    
    max_price = merged_df['MW'].max()
    min_price = merged_df['MW'].min()
    
    max_production = results_df[['Wind Power Generated']].max().max()
    min_production = results_df[['Wind Power Generated']].min().min()
    
    cost = 0  
    profit=total_revenue-cost
    increase_in_profit = total_revenue-unoptimized_revenue- cost  

    pd.options.display.float_format = '{:,.2f}'.format

    summary = pd.DataFrame({
        "Metric": ["Total Production (MWh)", "Max Production (MWh)", "Min Production (MWh)", "Max Price ($/MWh)", "Min Price ($/MWh)","Revenue Change from Baseline ($)","Profit ($)"],
        "Value": [total_production, max_production, min_production, max_price, min_price, increase_in_profit,profit]
    })

    return results_df, summary



Example Implementation

In [7]:
# Define parameters
params = {
    'power_at_height': 'power at 100m (MW)', 
    'private_market_demand': .5,
    'private_price': 75,
    'storage_efficiency': 0.9,
    'charge_C_rate': 0.5,
    'discharge_C_rate': 1,
    'n': 100,
    'battery_power_cost_per_kw': 100,  # Cost per kW of battery power capacity
    'battery_energy_cost_per_kwh': 100,  # Cost per kWh of battery energy capacity
    'discount_rate': 0.05,  # Discount rate for NPV calculation
    'degradation_cost_per_cycle': 20,  # Cost per cycle of battery degradation
}
max_capacity=10

    # Optimize for power at 100m
results_df,summary = optimize_energy(merged_df, params,max_capacity)

    # Switch to 200m
results_200m = optimize_energy(merged_df, params,max_capacity) 


Result Dataframes

In [8]:
results_df[0:40]

,Hour,Energy Sold to Private Market,Energy Discharged to Private Market,Energy Sold to Grid,Energy Charged,Energy Discharged to Grid,Energy Stored,Wind Power Generated,LMP DAM Price
0,0,0.50,0.00,237.57,0.00,0.00,0.00,238.07,66.65
1,1,0.50,0.00,177.80,1.11,0.00,1.00,179.41,62.66
2,2,0.50,0.00,142.33,0.00,0.00,1.00,142.83,64.72
3,3,0.50,0.00,94.84,5.00,0.00,5.50,100.34,60.83
4,4,0.50,0.00,68.22,5.00,0.00,10.00,73.72,61.59
5,5,0.50,0.00,55.76,0.00,0.00,10.00,56.26,63.78
6,6,0.50,0.00,53.47,0.00,5.00,5.00,53.97,79.10
7,7,0.50,0.00,61.55,0.00,2.50,2.50,62.05,87.81
8,8,0.50,0.00,34.91,0.00,1.25,1.25,35.41,81.11
9,9,0.50,0.00,17.75,0.00,0.62,0.62,18.25,59.49


In [9]:
summary

,Metric,Value
0,Total Production (MWh),"2,573,675.35"
1,Max Production (MWh),"3,025.82"
2,Min Production (MWh),0.00
3,Max Price ($/MWh),"1,336.97"
4,Min Price ($/MWh),-4.70
5,Revenue Change from Baseline ($),"307,282.02"
6,Profit ($),"222,135,123.34"


Validating Constraint Adherence 

In [10]:
class TestOptimizeEnergy(unittest.TestCase):
    def validate_constraints(self, results, params,storage_capacity):
        errors = []

        # Extract parameters
        wind_power_col = params["power_at_height"]
        wind_power = results['Wind Power Generated'].values

        storage_efficiency = params["storage_efficiency"]
        charge_C_rate = params["charge_C_rate"]
        discharge_C_rate = params["discharge_C_rate"]
        private_market_demand = params["private_market_demand"]
        dam_prices = results["LMP DAM Price"].values  # Ensure the correct column name for DAM prices

        # Loop through each hour
        for h in range(len(results)):
            sell_private = results["Energy Sold to Private Market"][h]
            sell_grid = results["Energy Sold to Grid"][h]
            charge = results["Energy Charged"][h]
            discharge_grid = results["Energy Discharged to Grid"][h]
            discharge_private = results["Energy Discharged to Private Market"][h]
            stored_energy = results["Energy Stored"][h]

            if h > 0:
                prev_stored_energy = results["Energy Stored"][h - 1]
            else:
                prev_stored_energy = 0

            # Constraint: Sell to grid ≤ excess energy
            if sell_grid > (wind_power[h] - sell_private + discharge_grid + .001):
                errors.append(f"Hour {h}: sell_grid exceeds available energy.")

            # Constraint: Charge ≤ excess energy
            if charge > (wind_power[h] - sell_private - sell_grid + 0.001):
                errors.append(f"Hour {h}: charge exceeds available energy.")

            # Constraint: Discharge ≤ stored energy
            if discharge_grid + discharge_private > stored_energy + .001:
                errors.append(f"Hour {h}: discharge exceeds stored energy.")

            # Constraint: Storage dynamics
            expected_stored_energy = prev_stored_energy + (charge * storage_efficiency) - discharge_grid - discharge_private
            if not np.isclose(stored_energy, expected_stored_energy, atol=.001):
                errors.append(f"Hour {h}: stored_energy mismatch (expected {expected_stored_energy}, got {stored_energy}).")

            # Constraint: Private market demand
            if not np.isclose(sell_private + discharge_private, private_market_demand, atol=.001):
                errors.append(f"Hour {h}: sell_private + discharge_private does not meet private market demand.")

            # Constraint: Stored energy ≤ storage capacity
            if stored_energy > storage_capacity + .001:
                errors.append(f"Hour {h}: stored_energy exceeds storage capacity.")

            # Constraint: Charge ≤ charge_C_rate * storage_capacity
            if charge > (charge_C_rate * storage_capacity + .001):
                errors.append(f"Hour {h}: charge exceeds maximum charge rate.")

            # Constraint: Discharge ≤ discharge_C_rate * storage_capacity
            if discharge_grid + discharge_private > (discharge_C_rate * storage_capacity + .001):
                errors.append(f"Hour {h}: discharge exceeds maximum discharge rate.")

        return errors

In [11]:
def test_constraints(self):
      # Run the validation function
      errors = self.validate_constraints(results_df, params)

      # Assert no errors
      if errors:
          for error in errors:
              print(error)
      self.assertEqual(len(errors), 0, "Constraint violations detected!")

In [12]:
errors = TestOptimizeEnergy().validate_constraints(results_df, params,max_capacity)
print(errors)  # Prints the errors or empty list if no violations


[]


 Implementation

**Battery Storage Optimization**:


Batteries generate revenue by shifting energy from periods of low prices, to periods of high prices, such as during peak demand. Profitability relies on storage capacity: even if additional wind turbines are installed, profitability will not increase without sufficient battery storage.The battery storage optimization algorithm is designed to determine the optimal amount of battery storage to maximize profitability. 


In [13]:
def calculate_payback_period(cash_flows, initial_investment):
    cumulative_cash_flow = np.cumsum(cash_flows) - initial_investment
    payback_period = next((year for year, cash in enumerate(cumulative_cash_flow) if cash > 0), np.inf)
    return payback_period

In [14]:
#Determine Range of Storage Capacity Options
storage_range=(30,300)
step_size=10

In [15]:
# Constants for keys used in the summary DataFrame
SUMMARY_METRIC_KEY = 'Metric'
SUMMARY_VALUE_KEY = 'Value'

def calculate_metrics_for_capacity(capacity: float, merged_df: pd.DataFrame, params: Dict) -> Dict:
    """
    Calculate financial metrics for a given storage capacity.

    Parameters:
    - capacity (float): The storage capacity to evaluate.
    - merged_df (pd.DataFrame): The input data for the optimization model.
    - params (dict): Model parameters including costs, revenue, and degradation data.

    Returns:
    - dict: Financial metrics for the given storage capacity.
    """

    def calculate_battery_costs(capacity: float, params: Dict) -> float:
        """Calculate total battery costs (power + energy)."""
        power_cost = params['battery_power_cost_per_kw'] * capacity * 1000
        energy_cost = params['battery_energy_cost_per_kwh'] * capacity
        return power_cost + energy_cost

    def calculate_degradation_cost(results_df: pd.DataFrame, capacity: float, params: Dict) -> float:
        """Calculate degradation cost based on cycles used."""
        charge_discharge_sum = (
            results_df['Energy Charged']
            + results_df['Energy Discharged to Grid']
            + results_df['Energy Discharged to Private Market']
        )
        cycles_used = charge_discharge_sum.sum() / capacity
        return cycles_used * params['degradation_cost_per_cycle']

    def npv_calculation(net_profit: float, discount_rate: float, periods: int) -> float:
        """Calculate Net Present Value (NPV)."""
        return net_profit / (1 + discount_rate) ** np.arange(periods).sum()

    def calculate_unoptimized_revenue(results_df: pd.DataFrame, params: Dict) -> float:
        """Calculate baseline revenue without optimization."""
        wind_excess = results_df["Wind Power Generated"] - params['private_market_demand']
        wind_revenue = (wind_excess.clip(lower=0) * results_df["LMP DAM Price"]).sum()
        return params['private_market_demand'] * params['private_price'] + wind_revenue

    # Update parameters with the current storage capacity
    params['storage_capacity'] = capacity

    # Run optimization model
    results_df, summary = optimize_energy(merged_df, params, capacity)
    total_revenue = summary.loc[summary[SUMMARY_METRIC_KEY] == 'Profit ($)', SUMMARY_VALUE_KEY].values[0]

    # Calculate costs and financial metrics
    total_battery_cost = calculate_battery_costs(capacity, params)
    degradation_cost = calculate_degradation_cost(results_df, capacity, params)
    unoptimized_revenue = calculate_unoptimized_revenue(results_df, params)
    net_profit = total_revenue - total_battery_cost - degradation_cost
    npv = npv_calculation(net_profit, params['discount_rate'], len(results_df))
    payback_period = calculate_payback_period(
        net_profit - unoptimized_revenue, total_battery_cost
    )

    # Return raw numeric results (defer string formatting)
    return {
        'Storage Capacity (MWh)': capacity,
        'Profit ($)': net_profit,
        'NPV ($)': npv,
        'Payback Period (years)': payback_period,
        'Increase in Profit ($)': net_profit - unoptimized_revenue,
        'Battery Cost ($)': total_battery_cost,
        "Total Revenue ($)": total_revenue,
        "Degradation Cost ($)": degradation_cost,
    }

def find_best_storage_capacity(merged_df: pd.DataFrame, params: Dict, storage_range: Tuple[float, float], step_size: float,
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Find the optimal storage capacity for maximizing profit.

    Parameters:
    - merged_df (pd.DataFrame): The input data for the optimization model.
    - params (dict): Model parameters including costs, revenue, and degradation data.
    - storage_range (tuple): Range of storage capacities to test (min, max).
    - step_size (float): Increment for testing storage capacities.

    Returns:
    - Tuple[pd.DataFrame, pd.DataFrame]: Results for all capacities and the optimal configuration.
    """
    min_capacity, max_capacity = storage_range
    storage_options = np.arange(min_capacity, max_capacity + step_size, step_size)

    # Parallelize the processing of storage capacities
    results: list[Dict] = []
    with ThreadPoolExecutor() as executor:
        futures = {
            executor.submit(calculate_metrics_for_capacity, capacity, merged_df, params): capacity
            for capacity in storage_options
        }
        for future in futures:
            try:
                results.append(future.result())
            except Exception as e:
                print(f"Error calculating metrics for capacity {futures[future]}: {e}")

    if not results:
        raise ValueError("No valid configurations were found. Check input data and parameters.")

    # Convert results to a DataFrame
    battery_results = pd.DataFrame(results)

    # Find the optimal storage capacity
    optimal_idx = battery_results['Increase in Profit ($)'].idxmax()
    optimal_config = battery_results.loc[optimal_idx]

    optimal_config_df = pd.DataFrame([{
        'Optimal Storage Capacity (MWh)': optimal_config['Storage Capacity (MWh)'],
        'Profit ($)': f"{optimal_config['Profit ($)']:,.2f}",
        'NPV ($)': f"{optimal_config['NPV ($)']:,.2f}",
        'Payback Period (years)': f"{optimal_config['Payback Period (years)']:.2f}",
    }])

    return battery_results, optimal_config_df


Example Usage

In [16]:
battery_results, optimal_config_df = find_best_storage_capacity(merged_df, params, storage_range, step_size)

C:\Users\Rina\AppData\Local\Temp\ipykernel_20832\1292512766.py:36: RuntimeWarning: overflow encountered in power
  return net_profit / (1 + discount_rate) ** np.arange(periods).sum()
C:\Users\Rina\AppData\Local\Temp\ipykernel_20832\1292512766.py:36: RuntimeWarning: overflow encountered in power
  return net_profit / (1 + discount_rate) ** np.arange(periods).sum()
C:\Users\Rina\AppData\Local\Temp\ipykernel_20832\1292512766.py:36: RuntimeWarning: overflow encountered in power
  return net_profit / (1 + discount_rate) ** np.arange(periods).sum()
C:\Users\Rina\AppData\Local\Temp\ipykernel_20832\1292512766.py:36: RuntimeWarning: overflow encountered in power
  return net_profit / (1 + discount_rate) ** np.arange(periods).sum()
C:\Users\Rina\AppData\Local\Temp\ipykernel_20832\1292512766.py:36: RuntimeWarning: overflow encountered in power
  return net_profit / (1 + discount_rate) ** np.arange(periods).sum()
C:\Users\Rina\AppData\Local\Temp\ipykernel_20832\1292512766.py:36: RuntimeWarning: ov

Results

In [17]:
battery_results

,Storage Capacity (MWh),Profit ($),NPV ($),Payback Period (years),Increase in Profit ($),Battery Cost ($),Total Revenue ($),Degradation Cost ($)
0,30,"219,672,463.91",0.00,inf,"-1,837,714.91",3003000,"222,698,071.48","22,607.57"
1,40,"218,947,211.50",0.00,inf,"-2,562,967.32",4004000,"222,973,588.27","22,376.76"
2,50,"218,219,440.69",0.00,inf,"-3,290,738.13",5005000,"223,246,656.98","22,216.29"
3,60,"217,489,095.97",0.00,inf,"-4,021,082.85",6006000,"223,517,159.52","22,063.55"
4,70,"216,756,354.86",0.00,inf,"-4,753,823.96",7007000,"223,785,259.26","21,904.41"
5,80,"216,021,473.77",0.00,inf,"-5,488,705.05",8008000,"224,051,236.31","21,762.54"
6,90,"215,283,858.72",0.00,inf,"-6,226,320.10",9009000,"224,314,493.05","21,634.33"
7,100,"214,543,852.01",0.00,inf,"-6,966,326.81",10010000,"224,575,342.90","21,490.89"
8,110,"213,802,159.95",0.00,inf,"-7,708,018.87",11011000,"224,834,510.75","21,350.81"
9,120,"213,058,798.82",0.00,inf,"-8,451,380.00",12012000,"225,092,007.10","21,208.28"


In [17]:
optimal_config_df

,Optimal Storage Capacity (MWh),Profit ($),NPV ($),Payback Period (years)
0,30.00,"219,672,463.91",0.00,inf
